# JB4 CIMR Double Remap L1b-->L1r-->L1c

This notebook describes how to do a double remap for L1b to L1r to L1c. The user is free to play with different algorithms and other parameters. The default for this notebook is a CIMR remap from Ka BAND to L band, followed by a remap to an EASE2 36km grid. We advise the user to follow the steps for this default configuration first, after which they can play with the parameters. 

You will need to enter the path to the L1b data in the configuration file, which is also saved in this folder. The output location will also need to be specified in the configuration file. 

# Step 1 - Import relevant modules

In [1]:
# Import Relevant Modules
from cimr_rgb.config_file import ConfigFile
from cimr_rgb.data_ingestion import DataIngestion
from cimr_rgb.regridder import ReGridder
from cimr_rgb.grid_generator import GRIDS
from cimr_rgb.product_generator import ProductGenerator

import copy
from numpy import full, nan, nanmean, isnan, isinf, polyfit

# You will need to configure matplotlib for your machine, you can probably remove the two lines below. 
import matplotlib # Remove this is plotting issues. 
tkagg = matplotlib.use('TkAgg') # Remove this if plotting issues. 
import matplotlib.pyplot as plt
plt.ion()

# Step 2 - Add your paths to the configuration file, load the configuration file and validate the configuration file

In [2]:
# Define path for configuration file of the first regrid.
config_1_path = '/home/beywood/Desktop/MS3/CIMR-RGB/notebooks/L1b_L1r_L1c/l1bl1rl1c_config.xml'

# Validate your config file
config_object_L1b2L1r = ConfigFile(config_1_path)

/home/beywood/ST/CIMR_RGB/CIMR-RGB/dpr/L1B/CIMR/SCEPS_l1b_sceps_geo_central_america_scene_1_unfiltered_tot_minimal_nom_nedt_apc_tot_v2p1.nc


In [3]:
# You can inspect the parameters in the config file. There are additional parameters relevant to your particular configuration
# that are added as part of the validation. 
for attr, value in config_object_L1b2L1r.__dict__.items():
    print(f"{attr}: {value}")

output_path: /home/beywood/Desktop/MS3/CIMR-RGB/notebooks/L1b_L1r_L1c
timestamp: 2024-12-10_12-55-58
timestamp_fmt: %Y-%m-%d_%H-%M-%S
logpar_config: {'version': 1, 'disable_existing_loggers': False, 'loggers': {'rgb-logger': {'level': 'INFO', 'handlers': ['stdout', 'file'], 'propagate': False}}, 'handlers': {'stdout': {'class': 'logging.StreamHandler', 'formatter': 'simple', 'stream': 'ext://sys.stdout', 'level': 'INFO'}, 'file': {'class': 'logging.FileHandler', 'level': 'INFO', 'formatter': 'balanced', 'filename': PosixPath('/home/beywood/Desktop/MS3/CIMR-RGB/notebooks/L1b_L1r_L1c/logs/l1bl1rl1c_config.log'), 'mode': 'w'}}, 'formatters': {'simple': {'format': '[%(name)s - %(levelname)s]: %(message)s'}, 'simple2': {'format': '[%(name)s - %(levelname)s | %(module)s - L%(lineno)d]: %(message)s'}, 'advanced': {'format': '[%(levelname)s][%(name)s][%(module)s][%(funcName)s][L%(lineno)d][%(asctime)s]: %(message)s', 'datefmt': '%Y-%m-%dT%H:%M:%S%z'}, 'balanced': {'format': '[%(asctime)s][%(le

# Step 3 - Ingest L1b data

In [4]:
# Ingest your data into the CIMR-RGB data dictionary format. 

l1b_data_dict = DataIngestion(config_object_L1b2L1r).ingest_data()  
    

[rgb-logger - INFO]: `combine_cimr_feeds`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `combine_cimr_feeds` -- Started Execution
[rgb-logger - INFO]: `combine_cimr_feeds` -- Executed in: 0.00s
[rgb-logger - INFO]: `combine_cimr_feeds` -- CPU User Time (Change): 0.01s
[rgb-logger - INFO]: `combine_cimr_feeds` -- CPU System Time: 0.00s
[rgb-logger - INFO]: `combine_cimr_feeds` -- CPU Total Time: 0.01s
[rgb-logger - INFO]: `combine_cimr_feeds` -- Process-Specific CPU Usage (Before): 0.00%
[rgb-logger - INFO]: `combine_cimr_feeds` -- Process-Specific CPU Usage (After): 265.50%
[rgb-logger - INFO]: `combine_cimr_feeds` -- Memory Usage Change: 1.718750 MB
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `combine_cimr_feeds`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `combine_cimr_feeds` -- Started Execution
[rgb-logger - INFO]: `combine_cimr_feeds` -- Executed in: 0.33s
[rgb-logger - INFO]: `combine_cimr_feeds` -- CPU User Time (Change):

# Step 4 - Perform the first regrid and Inspect the result

In [5]:
# Regrid L1b to L1r

l1r_data_dict = ReGridder(config_object_L1b2L1r).regrid_data(l1b_data_dict)

[rgb-logger - INFO]: `get_grid`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `get_grid` -- Started Execution
[rgb-logger - INFO]: `get_grid` -- Executed in: 0.00s
[rgb-logger - INFO]: `get_grid` -- CPU User Time (Change): 0.00s
[rgb-logger - INFO]: `get_grid` -- CPU System Time: 0.00s
[rgb-logger - INFO]: `get_grid` -- CPU Total Time: 0.00s
[rgb-logger - INFO]: `get_grid` -- Process-Specific CPU Usage (Before): 0.00%
[rgb-logger - INFO]: `get_grid` -- Process-Specific CPU Usage (After): 0.00%
[rgb-logger - INFO]: `get_grid` -- Memory Usage Change: 0.000000 MB
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: Regridding band: `KA`
[rgb-logger - INFO]: `sample_selection`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `sample_selection` -- Started Execution
[rgb-logger - INFO]: `get_l1c_samples`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `get_l1c_samples` -- Started Execution
[rgb-logger - INFO]: `get_neighbours`
[rgb-

In [6]:
# Inspect variables

source_band_data = l1b_data_dict[config_object_L1b2L1r.source_band[0]]
target_band_data = l1b_data_dict[config_object_L1b2L1r.target_band[0]]

 
for variable in source_band_data.keys():
    print(f"{variable}, shape = {source_band_data[variable].shape}")

longitude_fore, shape = (3077216,)
longitude_aft, shape = (3076624,)
processing_scan_angle_fore, shape = (3077216,)
processing_scan_angle_aft, shape = (3076624,)
latitude_fore, shape = (3077216,)
latitude_aft, shape = (3076624,)
nedt_4_fore, shape = (3077216,)
nedt_4_aft, shape = (3076624,)
acq_time_utc_fore, shape = (3077216,)
acq_time_utc_aft, shape = (3076624,)
nedt_v_fore, shape = (3077216,)
nedt_v_aft, shape = (3076624,)
bt_h_fore, shape = (3077216,)
bt_h_aft, shape = (3076624,)
azimuth_fore, shape = (3077216,)
azimuth_aft, shape = (3076624,)
oza_fore, shape = (3077216,)
oza_aft, shape = (3076624,)
bt_4_fore, shape = (3077216,)
bt_4_aft, shape = (3076624,)
nedt_3_fore, shape = (3077216,)
nedt_3_aft, shape = (3076624,)
bt_3_fore, shape = (3077216,)
bt_3_aft, shape = (3076624,)
bt_v_fore, shape = (3077216,)
bt_v_aft, shape = (3076624,)
nedt_h_fore, shape = (3077216,)
nedt_h_aft, shape = (3076624,)
feed_horn_number_fore, shape = (3077216,)
feed_horn_number_aft, shape = (3076624,)
sca

In [7]:
# Have a look at the L1r output 
fig, axs = plt.subplots(2, 1)
axs[0].imshow(l1r_data_dict['KA']['bt_h_fore'])
axs[0].set_title('bt_h_fore [KA band --> L band]')
axs[0].set_xlabel('Earth Sample Number [-]')
axs[0].set_ylabel('Scan Number [-]')
axs[1].imshow(l1r_data_dict['KA']['bt_h_aft'])
axs[1].set_title('bt_h_aft [KA band --> L band]')
axs[1].set_xlabel('Earth Sample Number [-]')
axs[1].set_ylabel('Scan Number [-]')

Text(0, 0.5, 'Scan Number [-]')

# Step 5 - Prepare data for the second regrid (L1r-->L1c)

In [8]:
# Create the configuration for the next regrid
config_object_L1r2L1c = copy.deepcopy(config_object_L1b2L1r)
config_object_L1r2L1c.grid_type = 'L1C'
config_object_L1r2L1c.grid_definition = 'EASE2_G36km'
config_object_L1r2L1c.projection_definition = 'G'

In [9]:
# Prep your data, using the newly created config object
prepped_l1r_data_dict = DataIngestion(config_object_L1r2L1c).prepare_l1r_for_l1c(l1r_data_dict)

[rgb-logger - INFO]: `GridGenerator`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `GridGenerator` -- Started Execution
[rgb-logger - INFO]: `GridGenerator` -- Executed in: 0.00s
[rgb-logger - INFO]: `GridGenerator` -- CPU User Time (Change): 0.00s
[rgb-logger - INFO]: `GridGenerator` -- CPU System Time: 0.00s
[rgb-logger - INFO]: `GridGenerator` -- CPU Total Time: 0.00s
[rgb-logger - INFO]: `GridGenerator` -- Process-Specific CPU Usage (Before): 0.00%
[rgb-logger - INFO]: `GridGenerator` -- Process-Specific CPU Usage (After): 0.00%
[rgb-logger - INFO]: `GridGenerator` -- Memory Usage Change: 0.000000 MB
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `lonlat_to_xy_cea`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `lonlat_to_xy_cea` -- Started Execution
[rgb-logger - INFO]: `lonlat_to_xy_cea` -- Executed in: 0.00s
[rgb-logger - INFO]: `lonlat_to_xy_cea` -- CPU User Time (Change): 0.01s
[rgb-logger - INFO]: `lonlat_to_xy_cea` -- CPU S

# Step 6 - Perform the second regrid and inspect the result. 

In [10]:
# Perform the second regrid, using the new config object
l1c_data_dict = ReGridder(config_object_L1r2L1c).regrid_data(prepped_l1r_data_dict)

[rgb-logger - INFO]: `get_grid`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `get_grid` -- Started Execution
[rgb-logger - INFO]: `generate_grid_xy`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `generate_grid_xy` -- Started Execution
[rgb-logger - INFO]: `generate_grid_xy_ease2`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `generate_grid_xy_ease2` -- Started Execution
[rgb-logger - INFO]: `generate_grid_xy_ease2` -- Executed in: 0.00s
[rgb-logger - INFO]: `generate_grid_xy_ease2` -- CPU User Time (Change): 0.00s
[rgb-logger - INFO]: `generate_grid_xy_ease2` -- CPU System Time: 0.00s
[rgb-logger - INFO]: `generate_grid_xy_ease2` -- CPU Total Time: 0.00s
[rgb-logger - INFO]: `generate_grid_xy_ease2` -- Process-Specific CPU Usage (Before): 0.00%
[rgb-logger - INFO]: `generate_grid_xy_ease2` -- Process-Specific CPU Usage (After): 0.00%
[rgb-logger - INFO]: `generate_grid_xy_ease2` -- Memory Usage Change: 0.000000 MB
[rgb-logger - INF

In [11]:
# Inspect the result by reconstructing the chosen grid from cell_row and cell_col
grid_shape = GRIDS[config_object_L1r2L1c.grid_definition]['n_rows'], GRIDS[config_object_L1r2L1c.grid_definition]['n_cols']

# create nan array with shape of grid_shape
grid_fore, grid_aft = full(grid_shape, nan), full(grid_shape, nan)
variable_fore = l1c_data_dict['KA']['bt_h_fore']
cell_row_fore = l1c_data_dict['KA']['cell_row_fore']
cell_col_fore = l1c_data_dict['KA']['cell_col_fore']
variable_aft = l1c_data_dict['KA']['bt_h_aft']
cell_row_aft = l1c_data_dict['KA']['cell_row_aft']
cell_col_aft = l1c_data_dict['KA']['cell_col_aft']

for i in range(len(cell_row_fore)):
    grid_fore[cell_row_fore[i], cell_col_fore[i]] = variable_fore[i]

for i in range(len(cell_row_aft)):
    grid_aft[cell_row_aft[i], cell_col_aft[i]] = variable_aft[i]

fig, axs = plt.subplots(1, 2)
axs[0].set_title("bt_h_fore [Ka band --> L band --> EASEG36km]")
axs[0].set_xlabel("EASE2_G36km Grid Cols")
axs[0].set_ylabel("EASE2_G36km Grid Rows")
axs[1].set_title("bt_h_aft [Ka band --> L band --> EASEG36km]")
axs[1].set_xlabel("EASE2_G36km Grid Cols")
axs[1].set_ylabel("EASE2_G36km Grid Rows")
axs[0].imshow(grid_fore)
axs[1].imshow(grid_aft)
plt.tight_layout()
plt.show()

# Step 7 - Generate the L1c product and save to disk 

In [12]:
# Convert to netcdf and save to disk 
ProductGenerator(config_object_L1r2L1c).generate_product(l1c_data_dict)

[rgb-logger - INFO]: Creating group: Measurement
[rgb-logger - INFO]: Creating group: Navigation
[rgb-logger - INFO]: Creating group: Processing_flags
[rgb-logger - INFO]: Creating group: Quality_information


# Step 8 - Perform a direct regrid (L1b-->L1c) to compare with the double regrid 

In [13]:
# Copy configuration object for a third remap and print the attributes so we can see what we need to change
config_object_L1c_direct = copy.deepcopy(config_object_L1r2L1c)

# Print attributes
for attr, value in config_object_L1c_direct.__dict__.items():
    print(f"{attr}: {value}")

output_path: /home/beywood/Desktop/MS3/CIMR-RGB/notebooks/L1b_L1r_L1c
timestamp: 2024-12-10_12-55-58
timestamp_fmt: %Y-%m-%d_%H-%M-%S
logpar_config: {'version': 1, 'disable_existing_loggers': False, 'loggers': {'rgb-logger': {'level': 'INFO', 'handlers': ['stdout', 'file'], 'propagate': False}}, 'handlers': {'stdout': {'class': 'logging.StreamHandler', 'formatter': 'simple', 'stream': 'ext://sys.stdout', 'level': 'INFO'}, 'file': {'class': 'logging.FileHandler', 'level': 'INFO', 'formatter': 'balanced', 'filename': PosixPath('/home/beywood/Desktop/MS3/CIMR-RGB/notebooks/L1b_L1r_L1c/logs/l1bl1rl1c_config.log'), 'mode': 'w'}}, 'formatters': {'simple': {'format': '[%(name)s - %(levelname)s]: %(message)s'}, 'simple2': {'format': '[%(name)s - %(levelname)s | %(module)s - L%(lineno)d]: %(message)s'}, 'advanced': {'format': '[%(levelname)s][%(name)s][%(module)s][%(funcName)s][L%(lineno)d][%(asctime)s]: %(message)s', 'datefmt': '%Y-%m-%dT%H:%M:%S%z'}, 'balanced': {'format': '[%(asctime)s][%(le

In [14]:
# Implement required edits to config 
config_object_L1c_direct.target_band = ['KA']


In [15]:
# We now perform the same regridding steps as before to generate the direct L1b to L1c regrid. 

l1b_data_dict = DataIngestion(config_object_L1c_direct).ingest_data()
l1c_direct_data_dict = ReGridder(config_object_L1c_direct).regrid_data(l1b_data_dict)


[rgb-logger - INFO]: `combine_cimr_feeds`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `combine_cimr_feeds` -- Started Execution
[rgb-logger - INFO]: `combine_cimr_feeds` -- Executed in: 0.64s
[rgb-logger - INFO]: `combine_cimr_feeds` -- CPU User Time (Change): 0.17s
[rgb-logger - INFO]: `combine_cimr_feeds` -- CPU System Time: 0.48s
[rgb-logger - INFO]: `combine_cimr_feeds` -- CPU Total Time: 0.65s
[rgb-logger - INFO]: `combine_cimr_feeds` -- Process-Specific CPU Usage (Before): 0.00%
[rgb-logger - INFO]: `combine_cimr_feeds` -- Process-Specific CPU Usage (After): 100.90%
[rgb-logger - INFO]: `combine_cimr_feeds` -- Memory Usage Change: 140.976562 MB
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `remove_out_of_bounds`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `remove_out_of_bounds` -- Started Execution
[rgb-logger - INFO]: `GridGenerator`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `GridGenerator` -- Starte

In [16]:
# Create the grid for the double regrid 
grid_double = full(grid_shape, nan)
variable_fore = l1c_data_dict['KA']['bt_h_fore']
cell_row_fore = l1c_data_dict['KA']['cell_row_fore']
cell_col_fore = l1c_data_dict['KA']['cell_col_fore']

for i in range(len(cell_row_fore)):
    grid_double[cell_row_fore[i], cell_col_fore[i]] = variable_fore[i]

# Create the grid for the direct remap
grid_direct = full(grid_shape, nan)
variable_fore = l1c_direct_data_dict['KA']['bt_h_fore']
cell_row_fore = l1c_direct_data_dict['KA']['cell_row_fore']
cell_col_fore = l1c_direct_data_dict['KA']['cell_col_fore']

for i in range(len(cell_row_fore)):
    grid_direct[cell_row_fore[i], cell_col_fore[i]] = variable_fore[i]

    
# Create the map plot comparison
cmap = 'viridis'
# bt_h plt
fig, axs = plt.subplots(1, 3, constrained_layout=True)
im00 = axs[0].imshow(grid_double[70:200, 180:270], cmap=cmap)
axs[0].set_xlabel("EASE2_G36km Grid Cols")
axs[0].set_ylabel("EASE2_G36km Grid Rows")
fig.colorbar(im00, ax=axs[0])
axs[0].set_title('bt_h double remap [Ka --> L --> EASE2]')
im01 = axs[1].imshow(grid_direct[70:200, 180:270], cmap=cmap)
axs[1].set_xlabel("EASE2_G36km Grid Cols")
axs[1].set_ylabel("EASE2_G36km Grid Rows")
fig.colorbar(im01, ax=axs[1])
axs[1].set_title('bt_h direct remap [Ka -> EASE2]')
bt_h_diff = abs(grid_double - grid_direct)
im02 = axs[2].imshow(bt_h_diff[70:200, 180:270], cmap=cmap)
axs[2].set_xlabel("EASE2_G36km Grid Cols")
axs[2].set_ylabel("EASE2_G36km Grid Rows")
fig.colorbar(im02, ax=axs[2])
axs[2].set_title('Difference (bt_h)')

# Create the scatter plot comparison
def scatter_stats(x, y):
    mask = ~isnan(x) & ~isnan(y) & ~isinf(x) & ~isinf(y)
    x = x[mask]
    y = y[mask]
    m, b = polyfit(x, y, 1)
    y_fit = m * x + b

    # Calculate R^2
    ss_res = sum((y - y_fit) ** 2)
    ss_tot = sum((y - y.mean()) ** 2)
    r_squared = 1 - (ss_res / ss_tot)

    return x, y, m, b, y_fit, r_squared

x = grid_double.flatten()
y = grid_direct.flatten()
x_h_fore, y_h_fore, m_h_fore, b_h_fore, y_fit_h_fore, r_squared = scatter_stats(x, y)

fig, axs = plt.subplots()
axs.scatter(x_h_fore, y_h_fore)
axs.plot(x_h_fore, y_fit_h_fore, color='red')
axs.legend(title=f"$R^2 = {r_squared:.3f}$")
axs.set_title('bt_h double regrid vs direct regrid')
axs.set_xlabel('bt_h double [K]')
axs.set_ylabel('bt_h direct[K]')

[rgb-logger - WARNING]: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument. (File: /tmp/ipykernel_550337/3496934524.py, Line: 63)


Text(0, 0.5, 'bt_h direct[K]')

In [17]:
# Print Statistics 
# Calculate the average relative difference
fore_mean_diff = nanmean(bt_h_diff)
print(f"Average relative difference for bt_h: {fore_mean_diff} K")
# Calculate percentage Differences
fore_percent_diff = (fore_mean_diff / nanmean(grid_direct)) * 100
print(f"Average percentage difference for bt_h: {fore_percent_diff} %")

Average relative difference for bt_h: 1.3463177777389714 K
Average percentage difference for bt_h: 0.6630724691935559 %
